In [3]:
from utils import logger

ImportError: cannot import name 'logger' from 'utils' (C:\Users\User\anaconda3\lib\site-packages\utils\__init__.py)

### Source:

https://towardsdatascience.com/writing-your-first-generative-adversarial-network-with-keras-2d16fd8d4889

In [ ]:
# import tensorflow as tf
# tf.test.is_gpu_available(cuda_only=False, min_cuda_compute_capability=None)

# tf.config.list_physical_devices('GPU')

In [ ]:
%tensorflow_version 1.x
#import tensorflow as tf

from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation
from keras.layers.advanced_activations import LeakyReLU
from keras.models import Sequential, Model
from keras.optimizers import Adam

import matplotlib.pyplot as plt

import sys

import numpy as np

In [ ]:
from keras.datasets import mnist

In [ ]:
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation
from keras.layers.advanced_activations import LeakyReLU

from keras.models import Sequential, Model
from keras.optimizers import Adam

### Now we focus on building out our two models!

#### Enter Player 1!
##### Given and input of noise z the Generator produces and image. 

Here we initialize a sequential model and add all of our desired layers to the model. When you build your own GAN, I would encourage you to experiment with these layers. The above should simply serve as an introductory guide!

A lot of the terminology should seem somewhat familiar, as we briefly explained these above. There are however two new concepts I would like to briefly touch on.

Alpha — α is a hyperparameter which controls the underlying value to which the function saturates negatives network inputs.

Momentum — This is a technique used in Neural Networks to speed up the training speed and improve accuracy.

In [ ]:
def build_generator():
    model = Sequential()
    model.add(Dense(256, input_dim=latent_dim))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(512))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(1024))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(np.prod(img_shape), activation='tanh'))
    model.add(Reshape(img_shape))
    
    model.summary()
    
    noise = Input(shape=(latent_dim,))
    img = model(noise)
    
    return Model(noise, img)

#### Enter Player 2!
##### Given an input image the Discriminator outputs the likelihood of the image being real. 

There is not much new here, except that we return the validity. The validity is the Discriminator’s guess in respect to the input image’s likelihood of being real or not.

Additionally, some of you may have noticed that we use two different Activation functions for the last layer in each of the models. You can read more about these functions in a great post here.

In [ ]:
def build_discriminator():
    model = Sequential()
    model.add(Flatten(input_shape=img_shape))
    model.add(Dense(512))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(256))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(1, activation='sigmoid'))
    
    model.summary()
    
    img = Input(shape=img_shape)
    validity = model(img)
    
    return Model(img, validity)

#### Player 1 vs Player 2!

In [ ]:
'''
A helper function that we will use to track the training progress
'''
def sample_images(epoch):
    r, c = 5, 5
    noise = np.random.normal(0, 1, (r * c, latent_dim))
    gen_imgs = generator.predict(noise)
    
    gen_imgs = 0.5 * gen_imgs + 0.5
    
    fig, axs = plt.subplots(r, c)
    
    cnt = 0
    
    for i in range(r):
        for j in range(c):
            axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
            axs[i,j].axis('off')
            cnt += 1
    fig.savefig("images/%d.png" % epoch)
    plt.close()

#### Now that we have constructed our two models it’s time to pit them against each other.

In [ ]:
def train(epochs, batch_size=128, sample_interval=50):
    
    '''
    We do this by defining a training function, loading the data set, 
    re-scaling our training images and setting the ground truths.
    '''
    (X_train, _), (_, _) = mnist.load_data()
    
    X_train = X_train / 127.5 - 1.
    X_train = np.expand_dims(X_train, axis=3)
    
    valid = np.ones((batch_size, 1))
    fake = np.zeros((batch_size, 1))
    
    '''
    We then loop through a number of epochs to train our Discriminator by 
    first selecting a random batch of images from our true dataset, 
    generating a set of images from our Generator, 
    feeding both set of images into our Discriminator, 
    and finally setting the loss parameters for both the real and fake images, 
    as well as the combined loss.
    '''
    for epoch in range(epochs):
        idx = np.random.randint(0, X_train.shape[0], batch_size)
        imgs = X_train[idx]
        
        noise = np.random.normal(0, 1, (batch_size, latent_dim))
        
        gen_imgs = generator.predict(noise)
        
        d_loss_real = discriminator.train_on_batch(imgs, valid)
        d_loss_fake = discriminator.train_on_batch(gen_imgs, fake)
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
        

        '''
        And within the same loop we train our Generator, 
        by setting the input noise and ultimately training 
        the Generator to have the Discriminator label its samples 
        as valid by specifying the gradient loss.
        '''
        noise = np.random.normal(0, 1, (batch_size, latent_dim))

        g_loss = combined.train_on_batch(noise, valid)

        
        '''
        Additionally, in order for us to keep track of our training process, 
        we print the progress and save the sample image output depending on the epoch 
        interval specified.
        
        As you may have noticed, when the specific sample_interval is hit, 
        we call the sample_image function.
        '''
        print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" 
               % (epoch, d_loss[0], 100*d_loss[1], g_loss))

        if epoch % sample_interval == 0:
            sample_images(epoch)

Now the final piece to our puzzle is to simply call our original GAN class and pass in the expected parameters.

In [ ]:
#initialize our class
img_rows = 28
img_cols = 28
channels = 1
img_shape = (img_rows, img_cols, channels)
latent_dim = 100

#we will be using Adam as our optimizer function
optimizer = Adam(0.0002, 0.5)

'''
Here we build and compile our Discriminator, 
set its loss function, the optimizer we want to use, 
and the type of accuracy we want to measure against!
'''
discriminator = build_discriminator()
discriminator.compile(loss='binary_crossentropy',
    optimizer=optimizer,
    metrics=['accuracy'])

'''
This builds the Generator and defines the input noise. 
In a GAN the Generator network takes noise z as an input to produce its images.
'''
generator = build_generator()
z = Input(shape=(latent_dim,))
img = generator(z)

'''
This ensures that when we combine our networks we only train the Generator.
'''
discriminator.trainable = False

'''
This specifies that our Discriminator will take the images generated by 
our Generator and true dataset and set its output to a parameter called validity, 
which will indicate whether the input is real or not.
'''
validity = discriminator(img)

'''
Here we combined the models and also set our loss function and optimizer. 
The ultimate goal here is for the Generator to fool the Discriminator.
'''
combined = Model(z, validity)
combined.compile(loss='binary_crossentropy', optimizer=optimizer)

In [ ]:
train(epochs=100000, batch_size=132, sample_interval=10000)

In [ ]:
# with tf.compat.v1.variable_scope(tf.compat.v1.variable_scope(),reuse=False):
#     train(epochs=100000, batch_size=132, sample_interval=10000)

from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten
from keras.layers import BatchNormalization
from keras.layers.advanced_activations import LeakyReLU
from keras.models import Sequential, Model
from keras.optimizers import Adam
import matplotlib.pyplot as plt
import numpy as np


class GAN():
    def __init__(self):
        self.img_rows = 28
        self.img_cols = 28
        self.channels = 1
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.latent_dim = 100
        optimizer = Adam(0.0002, 0.5)
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='binary_crossentropy',
                                   optimizer=optimizer,
                                   metrics=['accuracy'])
        self.generator = self.build_generator()
        z = Input(shape=(self.latent_dim,))
        img = self.generator(z)
        self.discriminator.trainable = False
        validity = self.discriminator(img)
        self.combined = Model(z, validity)
        self.combined.compile(loss='binary_crossentropy', optimizer=optimizer)

    def build_generator(self):

        model = Sequential()

        model.add(Dense(256, input_dim=self.latent_dim))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(1024))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(np.prod(self.img_shape), activation='tanh'))
        model.add(Reshape(self.img_shape))
        model.summary()
        noise = Input(shape=(self.latent_dim,))
        img = model(noise)
        return Model(noise, img)

    def build_discriminator(self):
        model = Sequential()

        model.add(Flatten(input_shape=self.img_shape))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(256))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(1, activation='sigmoid'))
        model.summary()
        img = Input(shape=self.img_shape)
        validity = model(img)
        return Model(img, validity)

    def train(self, epochs, batch_size=128, sample_interval=50):
        (X_train, _), (_, _) = mnist.load_data()
        X_train = X_train / 127.5 - 1.
        X_train = np.expand_dims(X_train, axis=3)
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))

        for epoch in range(epochs):
            idx = np.random.randint(0, X_train.shape[0], batch_size)
            imgs = X_train[idx]
            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))
            g_loss = self.combined.train_on_batch(noise, valid)
            gen_imgs = self.generator.predict(noise)
            d_loss_real = self.discriminator.train_on_batch(imgs, valid)
            d_loss_fake = self.discriminator.train_on_batch(gen_imgs, fake)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

        print("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100 * d_loss[1], g_loss))
        if epoch % sample_interval == 0:
            self.sample_images(epoch)

    def sample_images(self, epoch):
        r, c = 5, 5
        noise = np.random.normal(0, 1, (r * c, self.latent_dim))
        gen_imgs = self.generator.predict(noise)
        gen_imgs = 0.5 * gen_imgs + 0.5
        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i, j].imshow(gen_imgs[cnt, :, :, 0], cmap='gray')
                axs[i, j].axis('off')
                cnt += 1
        fig.savefig("images/%d.png" % epoch)
        plt.close()



gan = GAN()










gan.train(epochs=100000, batch_size=132, sample_interval=10000)